### a) Start a thread running a Custom Persona (User)

In [1]:
import os

from gai.persona.persona_builder import PersonaBuilder
here = os.path.dirname(os.path.abspath(__name__))
import_dir=os.path.join(here,"../../../src/gai/mace/node/persona/data/Diana")
builder = PersonaBuilder()
builder = await builder.import_async(import_dir=import_dir)
persona = builder.build()
nats = "nats://nats01:4222"

from gai.mace.node.mace_server import MaceServer
node = await MaceServer.create(servers=nats, persona=persona,is_user=True)

from gai.mace.node.thread_manager import ThreadManagerSingleton
tm = ThreadManagerSingleton.GetInstance()
tm.run_thread("User", node.serve)

INFO     GaiNetNode.connect: Ready.
INFO     MaceServer.listen: Server is starting to serve.


### b) Test response

rollcall

In [10]:
from gai.mace.user.mace_client import MaceClient
from gai.lib.common.logging import getLogger
logger = getLogger(__name__)
user = await MaceClient.create(servers="nats://nats01:4222")
await user.subscribe()
await user.rollcall()

INFO     GaiNetNode.connect: Ready.
INFO     start rollcall
DEBUG    mace_client.rollcall_handler: {'Name': 'Diana', 'ClassName': 'Assistant', 'AgentDescription': 'Hi there! I am an Assistant class generative AI agent.', 'AgentShortDesc': None, 'Image64': '...', 'Image128': '...'}
DEBUG    mace_client.rollcall_handler: Diana
DEBUG    mace_client.rollcall_handler: {'Name': 'George', 'ClassName': 'Assistant', 'AgentDescription': 'Hi there! I am an Assistant class generative AI agent.', 'AgentShortDesc': None, 'Image64': '...', 'Image128': '...'}
DEBUG    mace_client.rollcall_handler: George
DEBUG    mace_client.rollcall_handler: {'Name': 'Sara', 'ClassName': 'Assistant', 'AgentDescription': 'Hi there! I am an Assistant class generative AI agent.', 'AgentShortDesc': None, 'Image64': '...', 'Image128': '...'}
DEBUG    mace_client.rollcall_handler: Sara
INFO     Diana:Hi there! I am an Assistant class generative AI agent.
INFO     George:Hi there! I am an Assistant class generative AI agent

DEBUG    mace_client.rollcall_handler: {'Name': 'Diana', 'ClassName': 'Assistant', 'AgentDescription': 'Hi there! I am an Assistant class generative AI agent.', 'AgentShortDesc': None, 'Image64': '...', 'Image128': '...'}
DEBUG    mace_client.rollcall_handler: Diana


direct message

In [ ]:
from gai.mace.user.mace_client import MaceClient
import json
async def on_chat(msg):
    print(json.loads(msg.data.decode())["Chunk"], end="", flush=True)
user = await MaceClient.create(servers="nats://nats01:4222")
await user.subscribe(async_chat_handler=on_chat)
res=await user.dialogue(msg="Roses are red. ",flow_diagram="User->>Sara")


INFO     GaiNetNode.connect: Ready.
INFO     
User:
Roses are red. 


None<Sara>Indeed, roses are red. I have encountered this information in my database. Is there anything else you would like to share or discuss? </Sara>

### c) Remove the Custom Persona

In [3]:
from gai.mace.node.thread_manager import ThreadManagerSingleton
tm = ThreadManagerSingleton.GetInstance()
tm.stop_thread("User")

Thread User marked to stop.
